# Theory Solving Exercises

This notebook contains exercises about Theory Solving with the answer set solver *clingo*.

Before you start, we recommend you to read sections 6 and 7 of [1].

[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/tplp/KaminskiRSW23.pdf)

# Exercise 1

In this exercise, you have to extend the script [solving.py](solving.py) to:
1. Print a table with information about every symbolic atom, and show the initial assignment
2. Profile the search of the solver
3. Let the user make the decisions of the solver

Consider this program:

In [2]:
! cat example.lp

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
dom(1..2).
{ a(X) } :- dom(X).
{ b(X) } :- dom(X).
c(X) :- b(X).
:- a(X), not c(X).
:- not a(X), c(X).


In [3]:
! clingo example.lp 0 -V0

dom(1) dom(2)
dom(1) dom(2) b(2) c(2) a(2)
dom(1) dom(2) b(1) c(1) a(1)
dom(1) dom(2) b(1) b(2) c(1) c(2) a(1) a(2)
SATISFIABLE


In the next cell we print the expected output of the command:
* `python solving.py example.lp`

The script is interactive (see the explanations below).
  
This interaction usually does not work on the notebook, 
so you will have to run the command on your console.

In [4]:
! cat output/output-01.txt

solving version 1.0
Reading from example.lp
symbol		is_fact		is_external	program_literal	solver_literal
------------------------------------------------------------------------------
dom(1)		True		False		1		1
dom(2)		True		False		2		1
a(1)		False		False		7		4
a(2)		False		False		8		5
b(1)		False		False		3		2
b(2)		False		False		4		3
c(1)		False		False		5		2
c(2)		False		False		6		3

Assignment: 
  decision_level: 0
  has_conflict: False
  is_total: False
  literals in the assignment: 1 2 3 4 5
  trail: 1

0:dom(1)_dom(2) (P)
Solving...
b(1)_c(1) b(2)_c(2) a(1) a(2) ? a(1)
1:  a(1) b(1)_c(1) (P)
b(2)_c(2) a(2) ? -a(2)
2:    -a(2) -b(2)_-c(2) (P)
2:    -a(2) -b(2)_-c(2) (C)
Answer: 1
dom(1) dom(2) b(1) c(1) a(1)
SATISFIABLE

Models       : 1+
Calls        : 1
Time         : 4.750s (Solving: 4.74s 1st Model: 4.74s Unsat: 0.00s)
CPU Time     : 0.006s

### Part 1

*Print a table with information about every symbolic atom, and show the initial assignment.*

As can be seen in the table, for every symbolic atom you should print information about:
* whether it is a fact and/or external
* its program literal and its solver literal

Observe how the trail of the initial assignment only has the solver literal `1`
(that represents all facts).

Some notes on the relation between symbolic atoms, program literals and solver literals:
* Every symbolic atom is associated with one program literal, 
and every program literal is associated with one solver literal.
* There may be program literals unrelated to any symbolic atom, 
and there may be solver literals unrelated to any program literal.
* In the current implementation of *clingo* (v5.7.0)
two different symbolic atoms are associated always with two different program literals, 
but this may change in some future version of *clingo*.
* Two program literals may be associated with the same solver literal.
You can see this in the table, where `b(1)` and `c(1)` have the same solver literal, 
as well as `b(2)` and `c(2)`.

You can implement this part modifying the methods `__init__` and `init` of the propagator at [solving.py](solving.py).

### Part 2

*Profile the search of the solver.*

In our example, we have first 
```
0:dom(1)_dom(2) (P)
```
This line indicates that when `propagate` is called the first time, 
the assignment is at decision level `0`,
and the atoms `dom(1)` and `dom(2)` are true.

These atoms are separated by `_` to show that they are associated with the same solver literal.

We skip the next two lines (we have a look at the second of them in Part 3 below).

In the next line we have:
```
1:  a(1) b(1)_c(1) (P)
```
This indicates that when `propagate` is called the next time, 
the assignment is at decision level `1`, 
and atoms `a(1)`, `b(1)` and `c(1)` have become true.

The first atom, `a(1)`, has been chosen by the solver heuristic.
After this, the solver literal shared by the other two atoms (number `2`, see in the table above) has been propagated
and both become true.

We skip again the next line, and in the next one we have something similar:
```
2:    -a(2) -b(2)_-c(2) (P)
```
Now at `propagate` the assignment is at decision level `2`, 
and the atoms `a(2)`, `b(2)` and `c(2)` have become false (the symbol `-` indicates this).

The next line for check `(C)` is the same, since nothing was propagated, 
and then the answer set found is shown.

To implement this, you should print the current assignment at the methods `propagate`, `undo` and `check`.

In the script, this can be done implementing the method `_print_current_level`, 
and initializing/updating some data structures at the methods `__init__` and `init`.

Since up to here you have not modified the `decide` method, 
your output may be different than above.
For example, we obtain this:
```
...
0:dom(1)_dom(2) (P)
Solving...
1:  -b(1)_-c(1) -a(1) (P)
2:    -a(2) -b(2)_-c(2) (P)
2:    -a(2) -b(2)_-c(2) (C)
Answer: 1
dom(1) dom(2)
...
```
In this case, *clingo* made first `b(1)` true (and `c(1)`), and then `a(2)` false.

### Part 3

*Let the user make the decisions of the solver.*

In the output above, after the propagation at level `0` and the `Solving...` message, the script printed:
```
b(1)_c(1) b(2)_c(2) a(1) a(2) ? a(1)
```
In the interaction, it first printed the atoms before the `?`, 
to indicate that at this point those atoms are unassigned.
Then, *we* entered the atom `a(1)`.

Similarly, in the line
```
b(2)_c(2) a(2) ? -a(2)
```
the script tells us that `b(2)`, `c(2)` and `a(2)` are unassigned, 
and we choose to make `a(2)` false.

In the script, this can be implemented modifying the method `decide`, and using Python function `input(...)`, for example.

### More examples

Now we can add this rule:

In [5]:
! cat one.lp

:- a(X), b(X).


Then, if we do 
```
python solving.py example.lp one.lp
```
instead of 
```
0:dom(1)_dom(2) (P)
Solving...
b(1)_c(1) b(2)_c(2) a(1) a(2) ? a(1)
1:  a(1) b(1)_c(1) (P)
b(2)_c(2) a(2) ? -a(2)
2:    -a(2) -b(2)_-c(2) (P)
2:    -a(2) -b(2)_-c(2) (C)
Answer: 1
dom(1) dom(2) b(1) c(1) a(1)
```
now we obtain this (choosing first `a(1)` and then `-a(2)`:
```
0:dom(1)_dom(2) (P)
Solving...
b(1)_c(1) b(2)_c(2) a(1) a(2) ? a(1)
0:dom(1)_dom(2) -a(1) -b(1)_-c(1) (P)
b(2)_c(2) a(2) ? -a(2)
1:  -a(2) -b(2)_-c(2) (P)
1:  -a(2) -b(2)_-c(2) (C)
Answer: 1
dom(1) dom(2)
```
See how after choosing `a(1)`, instead of going to decision level `1` as above, 
we go back to `0`: how is this possible?

* After setting `a(1)` to true, the solver can propagate 
`a(1)`, `b(1)` and `c(1)` to true as before. 
But now that we added constraint `:- a(X), b(X)`
it can also propagate `b(1)` to false and generate a conflict.

* In this situation, *clingo* learns a new nogood and goes back to level `0`.
Then, it propagates `a(1)`, `b(1)` and `c(1)` to false, and the search continues...

This example shows that 
whenever the solver's propagation leads to a conflict, 
we cannot see what literals where propagated at that step.
This is a limitation of our profiling.

Going one step further, if now we add this:

In [6]:
! cat two.lp

:- not a(X), not b(X), dom(X).


Then the command:
```
python solving.py example.lp one.lp two.lp
```
generates this (after we choose `a(1)`):
```
...
0:dom(1)_dom(2) (P)
Solving...
a(1) a(2) b(1)_c(1) b(2)_c(2) ? a(1)
UNSATISFIABLE
...
```
In this case, *clingo* again reaches a conflict after choosing `a(1)`, 
but now it proves `UNSATISFIABLE` right away.

# Exercise 2

The task of this exercise is to specialize the script
that you implemented in Exercise 1 to the Latin Square problem.

In the Latin Square problem of size `n` we have to:
* fill a square grid of `n` times `n` cells with numbers from `1` to `n` such that
* a number does not occur twice in a row, and
* a number does not occur twice in a column.

There are `12` Latin squares of size `3`. There are only `4` where the top-left cell has number `1`.

They can be represented by this encoding:

In [4]:
! cat latin.lp

number(1..3).

latin(1,1,1).

1 { latin(X,Y,N) : number(N) } 1 :- number(X), number(Y).
:- latin(X,Y1,N), latin(X,Y2,N), Y1 < Y2, number(X), number(N).
:- latin(X1,Y,N), latin(X2,Y,N), X1 < X2, number(Y), number(N).

#show latin/3.


The choice rule fills the grid, and 
the two integrity constraints check the conditions on rows and columns.

In [7]:
! clingo latin.lp -V0 0

latin(1,1,1) latin(3,2,1) latin(2,3,1) latin(3,1,2) latin(2,2,2) latin(1,3,2) latin(2,1,3) latin(1,2,3) latin(3,3,3)
latin(1,1,1) latin(2,2,1) latin(3,3,1) latin(3,1,2) latin(1,2,2) latin(2,3,2) latin(2,1,3) latin(3,2,3) latin(1,3,3)
latin(1,1,1) latin(2,2,1) latin(3,3,1) latin(2,1,2) latin(3,2,2) latin(1,3,2) latin(3,1,3) latin(1,2,3) latin(2,3,3)
latin(1,1,1) latin(3,2,1) latin(2,3,1) latin(2,1,2) latin(1,2,2) latin(3,3,2) latin(3,1,3) latin(2,2,3) latin(1,3,3)
SATISFIABLE


The answer sets correspond to the Latin squares
```
1 3 2     1 2 3    1 3 2    1 2 3
3 2 1     3 1 2    2 1 3    2 3 1
2 1 3     2 3 1    3 2 1    3 1 2
```

In this exercise:
* at every call to `propagate`, `undo` and `check`
* *the script should print the square represented by the true atoms of the assignment*
* instead of printing the assignment at the current decision level.

For this, you can modify the function `_print_current_level` 
to call the following function `_print_latin` with the list of `latin/3` atoms that are 
true in the assignment.

In [11]:
def _print_latin(items): 
    grid = [[None]*3 for _ in range(3)]
    for item in items:
        X, Y, N = map(int, item[6:-1].split(','))
        grid[X-1][Y-1] = N
    for row in grid:
        print(' '.join(str(cell) if cell else '.' for cell in row))

items = ['latin(1,1,1)', 
                          'latin(2,2,2)', 'latin(2,3,1)',
                          'latin(3,2,1)', 'latin(3,3,3)']
_print_latin(items)

1 . .
. 2 1
. 1 3


Once this is in place, one can use the script to play Latin Squares with the help of *clingo*:
the user makes the decisions, and *clingo* does the rest.

This is part of the expected output if we run
```
python solving.py latin.lp
```
and we choose first `latin(1,2,2)` and then `latin(2,1,2)`.

In [13]:
! cat outputs/output-02.txt

1 . .
. . .
. . .
(P)
Solving...
latin(2,2,1) latin(3,2,1) latin(2,3,1) latin(3,3,1) latin(2,1,2) latin(3,1,2) latin(2,2,2) latin(1,2,2) latin(3,2,2) latin(2,3,2) latin(1,3,2) latin(3,3,2) latin(2,1,3) latin(3,1,3) latin(2,2,3) latin(1,2,3) latin(3,2,3) latin(2,3,3) latin(1,3,3) latin(3,3,3) ? latin(1,2,2)
1 2 3
. . .
. . .
(P)
latin(2,2,1) latin(3,2,1) latin(2,3,1) latin(3,3,1) latin(2,1,2) latin(3,1,2) latin(2,3,2) latin(3,3,2) latin(2,1,3) latin(3,1,3) latin(2,2,3) latin(3,2,3) ? latin(2,1,2)
1 2 3
2 3 1
3 1 2
(P)
1 2 3
2 3 1
3 1 2
(C)
Answer: 1
latin(1,1,1) latin(3,2,1) latin(2,3,1) latin(2,1,2) latin(1,2,2) latin(3,3,2) latin(3,1,3) latin(2,2,3) latin(1,3,3)
SATISFIABLE


After making `latin(1,2,2)` true, *clingo* can propagate `latin(1,3,3)`. 

After making `latin(2,1,2)` true, *clingo* can propagate the rest of the values and fill in the square.

# Exercise 3

A propagator has *implicitly* a set of nogoods (or constraints).

Those nogoods are not generated from the input logic program.

Instead, whenever they are needed at a propagation or check step, the propagator generates them and adds them to *clingo*:
they are made explicit *on demand*.

This is very useful when the set of nogoods is huge and *clingo* would be too slow if it had to store those nogoods explicitly.
This is usually the case, for example, if the logic program represents big numbers.

For instance, the propagator for difference constraints of Section 7 of [1] 
handles theory atoms of the form `&diff(x - y) <= k`, that stand for equations `x - y <= k`:
* The propagator has *implicitly* one nogood
for every set of theory atoms that stand for an inconsistent set of equations.
* Whenever such a set of theory atoms becomes true, 
the propagator detects it, and adds the corresponding nogood that prohibits it.

In this exercise we consider the second integrity constraint of our Latin square encoding:
* `:- latin(X1,Y,N), latin(X2,Y,N), X1 < X2, number(Y), number(N).`

Inside clingo, this generates one nogood for each ground instance of the constraint:
```
:-latin(2,1,1),latin(1,1,1).
:-latin(3,1,1),latin(1,1,1).
:-latin(3,1,1),latin(2,1,1).
:-latin(2,2,1),latin(1,2,1).
:-latin(3,2,1),latin(1,2,1).
:-latin(3,2,1),latin(2,2,1).
:-latin(2,3,1),latin(1,3,1).
:-latin(3,3,1),latin(1,3,1).
:-latin(3,3,1),latin(2,3,1).
:-latin(2,1,2),latin(1,1,2).
:-latin(3,1,2),latin(1,1,2).
:-latin(3,1,2),latin(2,1,2).
:-latin(2,2,2),latin(1,2,2).
:-latin(3,2,2),latin(1,2,2).
:-latin(3,2,2),latin(2,2,2).
:-latin(2,3,2),latin(1,3,2).
:-latin(3,3,2),latin(1,3,2).
:-latin(3,3,2),latin(2,3,2).
:-latin(2,1,3),latin(1,1,3).
:-latin(3,1,3),latin(1,1,3).
:-latin(3,1,3),latin(2,1,3).
:-latin(2,2,3),latin(1,2,3).
:-latin(3,2,3),latin(1,2,3).
:-latin(3,2,3),latin(2,2,3).
:-latin(2,3,3),latin(1,3,3).
:-latin(3,3,3),latin(1,3,3).
:-latin(3,3,3),latin(2,3,3).
```

You should *comment* this constraint, so that those nogoods will no longer be stored explicitly inside *clingo*.

**Your task in this exercise is to extend the propagator of the script
to process those nogoods *implicitly*.**

**You should do it in 4 possible ways or versions.**

The specific version is selected by option `-type=[1..4]` (`--type=0` was used by default for the previous exercises)
and is represented by an integer variable:
* CHECK
* PROPAGATE_ON_CONFLICT
* PROPAGATE_IF_POSSIBLE
* PROPAGATE_IF_POSSIBLE_STATEFUL

To test the script you can check if using the propagator with the commented rule you obtain the same answer sets as with the original encoding (calling `python solving.py latin.lp 0 --type=[1..4]`).

To simplify your work:
* You may want to modify the `decide` step to return `abs(fallback)` right away, 
so that *clingo* always decides on its own, without interaction, to add some number to a cell.
* You may add nogoods using option `lock=True` so that added nogoods are never deleted by the solver, for example with this line
( where `nogood` is a variable that contains a list of literals):
  - `control.add_nogood(nogood, lock=True) and control.propagate()`
* You do not have to consider multi-threading, i.e., you do not have to use variable `thread_id`. 

### Version CHECK

In the method `check` the propagator should:
* check if *any number* occurs twice in a column
* and if that is the case, it should add the nogood that prohibits that specific case.

This may also require some changes in the `__init__` and `init` methods.

### Version PROPAGATE_ON_CONFLICT

In the `propagate` method the propagator should 
do the same as in the `check` method.

The method should not use variable `changes`.

### Version PROPAGATE_IF_POSSIBLE

In the `propagate` method the propagator should 
do the same as in the `check` method, 
but also:
* if *some number* `n` occurs in a column `c` and some literal `l` representing that `n` occurs in another cell of `c` *is unassigned*
* then the propagator should add the nogood that forbids the combination of:
  - the literal for `n` being at its current position, and
  - `l`.

The method should not use the variable `changes`.

### Version PROPAGATE_IF_POSSIBLE_STATEFUL (optional)

In the `propagate` method the propagator should:
* have the same functionality as in version PROPAGATE_IF_POSSIBLE
* but instead of checking the conditions for *any number*
* it should only check the conditions that are triggered
  by the literals in the variable `changes`
* and it should check those conditions using some data structure `d`
  that represents for each number `n` and column `c` which literal (if any)
  that assigns `n` to some cell in `c` is true

This version also requires to implement the method `undo` 
to mantain the data structure `d`.
For example, if the propagator had `d[n][c]=l` and `l` becomes unassigned, then it should set `d[n][c]` to `None`.

In a real system such a data structure is used
to improve the performance of the propagator.

The idea is that instead of having to check conditions over all possible literals, 
the propagator only has to check the conditions looking at the information of this data structure.

The price paid for this is that the data structure has to be kept always
up to date with the assignment (hence the need to implement the `undo` method).